<a href="https://colab.research.google.com/github/PatrickRaborn/DS-Unit-2-Linear-Models/blob/master/Copy_of_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [X] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [X] Do one-hot encoding of categorical features.
- [X] Do feature selection with `SelectKBest`.
- [X] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [X] Get mean absolute error for the test set.
- [X] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# 1. Wrangle the data

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [11]:
# import all the things
import pandas as pd
import pandas_profiling
df = pd.read_csv(DATA_PATH + 'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

In [35]:
def wrangle(filepath):
  #import data
  df = pd.read_csv(filepath,
                   parse_dates=['SALE DATE'],
                   index_col='SALE DATE')
# Change column names: replace spaces with underscores
  df.columns = [col.replace(' ', '_') for col in df]
# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
  df['SALE_PRICE'] = (df['SALE_PRICE'].str.replace('$','').str.replace('-','').str.replace(',','').astype(int))
  # also sale price  more than 100 thousand and less than 2 million.
  df = df[(df['SALE_PRICE'] <= 2000000) & (df['SALE_PRICE'] >= 100000) &
          (df['BUILDING_CLASS_CATEGORY'] =='01 ONE FAMILY DWELLINGS')]
# Get a list of the top 10 neighborhoods
  top10 = df['NEIGHBORHOOD'].value_counts()[:10].index
  # BOROUGH is a numeric column, but arguably should be a categorical feature, so convert it from a number to a string
  df['BOROUGH'] = df['BOROUGH'].astype(str)
# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
  df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
  # remove high cardinality 
  high_cardinality = [col for col in df.select_dtypes('object').columns if df[col].nunique() >= 12]
  df.drop(columns=high_cardinality,inplace=True)
  empty = ['EASE-MENT','APARTMENT_NUMBER']
  df.drop(columns=empty,axis=1,inplace=True)
  return df
df = wrangle(DATA_PATH + 'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

In [38]:
# makesure neighborhood has 11 unique values 
df['NEIGHBORHOOD'].value_counts()

OTHER                       2406
FLUSHING-NORTH                98
ST. ALBANS                    94
SPRINGFIELD GARDENS           85
SO. JAMAICA-BAISLEY PARK      82
QUEENS VILLAGE                76
GREAT KILLS                   75
SOUTH OZONE PARK              70
BAYSIDE                       61
LAURELTON                     60
FLUSHING-SOUTH                57
Name: NEIGHBORHOOD, dtype: int64

In [39]:
# check that sale price between 100000 and 2000000
print(df['SALE_PRICE'].min())
print(df['SALE_PRICE'].max())

100000
2000000


In [40]:
#check that building class catergory = 01 ONE FAMILY DWELLINGS
df['BUILDING_CLASS_CATEGORY'].value_counts()

01 ONE FAMILY DWELLINGS    3164
Name: BUILDING_CLASS_CATEGORY, dtype: int64

In [45]:
# check df info 1 last time and make everything is good to go
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3164 entries, 2019-01-01 to 2019-04-30
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3164 non-null   object 
 1   NEIGHBORHOOD                    3164 non-null   object 
 2   BUILDING_CLASS_CATEGORY         3164 non-null   object 
 3   TAX_CLASS_AT_PRESENT            3164 non-null   object 
 4   BLOCK                           3164 non-null   int64  
 5   LOT                             3164 non-null   int64  
 6   ZIP_CODE                        3164 non-null   float64
 7   RESIDENTIAL_UNITS               3164 non-null   float64
 8   COMMERCIAL_UNITS                3164 non-null   float64
 9   TOTAL_UNITS                     3164 non-null   float64
 10  GROSS_SQUARE_FEET               3164 non-null   float64
 11  YEAR_BUILT                      3164 non-null   float64
 12  TAX_CLASS_AT_TIM

In [47]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1,A9,550000
2019-01-01,4,QUEENS VILLAGE,01 ONE FAMILY DWELLINGS,1,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1,A1,200000
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,10461.0,1.0,0.0,1.0,2043.0,1925.0,1,A1,810000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,11226.0,1.0,0.0,1.0,2680.0,1899.0,1,A1,125000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,11203.0,1.0,0.0,1.0,1872.0,1940.0,1,A5,620000


In [44]:
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE'],
      dtype='object')

# 2. Split data

In [46]:
# Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
target = 'SALE_PRICE'
y = df[target]
X = df.drop(columns=target)


In [51]:
cutoff = '2019-04-01'
mask = X.index < cutoff
X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

Timestamp('2019-03-30 00:00:00')

# 3. One-hot encoding of categorical variables


In [60]:
# import one-hot encoder
from category_encoders  import OneHotEncoder
# instantiate
ohe = OneHotEncoder(use_cat_names=True)
# fit the transformer to TRAINING DATA
ohe.fit(X_train)
# transform training data
XT_train = ohe.transform(X_train)
XT_test = ohe.transform(X_test)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


# 4. Pick  best features with Select K Best
There is 15 total features, I've decided to pick the top 8


In [81]:
# import SelectKBest
from sklearn.feature_selection import SelectKBest
# instantiate 
skb = SelectKBest(k=10)
# fit on skb 
skb.fit(XT_train,y_train)
# transfrom on skb
XTT_train = skb.transform(XT_train)
XTT_test = skb.transform(XT_test)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 4 16 27 38] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

# 5. Ridge regression


In [82]:
# import the ridge regression model
from sklearn.linear_model import LinearRegression, Ridge

# ridge below
model_r = Ridge(alpha=1, normalize=True)
model_r.fit(XT_train,y_train)
# linear regression below 
model_lr = LinearRegression()
model_lr.fit(XT_train,y_train)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# 6. Check metrics


In [83]:
# baseline MAE 
y_pred = [y_train.mean()] * len(y_train)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

Baseline MAE: 216497.12357684076


In [84]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
# LR MAE with one-hot encoded cat
print('LR training MAE:', mean_absolute_error(y_train, model_lr.predict(XT_train)))
print('LR test MAE:', mean_absolute_error(y_test, model_lr.predict(XT_test)))
# model improved on baseline about 30% 

LR training MAE: 155359.30169845052
LR test MAE: 163260.56230680062


In [85]:
# LR MAE with K-best 
model_lr.fit(XTT_train,y_train)
print('LR training MAE:', mean_absolute_error(y_train, model_lr.predict(XTT_train)))
print('LR test MAE:', mean_absolute_error(y_test, model_lr.predict(XTT_test)))

LR training MAE: 190368.28960833684
LR test MAE: 185199.49927528854


In [88]:
# R MAE 
print('Ridge training MAE:', mean_absolute_error(y_train,model_r.predict(XT_train)))
print('Ridge training MAE:', mean_absolute_error(y_test,model_r.predict(XT_test)))

Ridge training MAE: 169533.39969064726
Ridge training MAE: 173989.43904591954


In [87]:
# R MAE with XTT 
# doesn't work lets ask why but I am done! 

print('Ridge training MAE:', mean_absolute_error(y_train,model_r.predict(XTT_train)))
print('Ridge training MAE:', mean_absolute_error(y_test,model_r.predict(XTT_test)))

ValueError: ignored